# Chapter 2: Environment File Config

The basis of OpenAI-Gym is the creation of a file called the <b>environment file</b>.

This <b>environment file</b> will define how the learning file communicates with the OpenAI-Gym API.
We will go over the environment file used for the turtlebot simulation throughout this entire course.

Just to be clear, you are going to use the **gym_gazebo** module, not the original gym module. The reason is that in order to use Gazebo, this add-on had to be created. 

First, let's see where you can find these files.

These files are part of the Python installation of the gym_gazebo module. So, the most desirable location to put them is in the **dist-packages** directory. In this case, you will find them in **"/usr/local/lib/python2.7/dist-packages/,"** inside the folder **gym_gazebo**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd /usr/local/lib/python2.7/dist-packages/gym_gazebo
ls

The relevant things for now are the <i>**\_\_init\_\_.py**</i> file and the <i>**envs**</i> folder.
<ul>
    <li>
    **\_\_init\_\_.py**: This file registers the different environments available in the gym infrastructure.<br>
    
    For the TurtleBot simulation, we set up different parameters; the most important ones being the **id** and the **timestep_limit**. The time step limit allows you to make the learning episodes shorter or longer, depending on your needs. For instance, if the action being performed is really short, decrease the time. On the other hand, if it's an action that needs more time to be performed properly, then increase the time step limit to allow the simulation to have time to perform the entire action.<br>
    <p style="background:#CD2122;color:white;">WARNING</p>
    <br>
    It's a very common error to put more time in the **timestep_limit** than afterwards, in the learning script maximum steps loop. This will result in the following error:<br><br>
    <span style="color:red;"><i><b>gym.error.Error: Tried to reset environment, which is not done</b></i><br></span>
    <br>
    So, always try to put the same value on both sides or more time on the learning script side. That way, it won't give this error that states that you have changed the episode without finishing. You have to make the timestep_limit equal or less than the total_time in the learning loop per episode. Never make the learning loop faster than the timestep_limit.
    </li>
</ul>

<ul>
    <li>
    **envs**: Inside this folder, you will find all the environment files defined. This is where you will have to put your own environment file for your simulation.
    </li>
</ul>

Get inside the <i>envs</i> folder and open the env file <b>gazebo_circuit_turtlebot_lidar.py</b>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd envs
vim gazebo_circuit_turtlebot_lidar.py

You will get something similar to this:

<p style="background:green;color:white;">**gazebo_circuit_turtlebot_lidar.py**</p>

In [ ]:
import gym
import rospy
import roslaunch
import time
import numpy as np

from gym import utils, spaces
from gym_gazebo.envs import gazebo_env
from geometry_msgs.msg import Twist
from std_srvs.srv import Empty

from sensor_msgs.msg import LaserScan

from gym.utils import seeding

class GazeboCircuitTurtlebotLidarEnv(gazebo_env.GazeboEnv):

    def __init__(self):
        # Launch the simulation with the given launchfile name
        gazebo_env.GazeboEnv.__init__(self, "GazeboCircuitTurtlebotLidar_v0.launch")
        self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=5)
        self.unpause = rospy.ServiceProxy('/gazebo/unpause_physics', Empty)
        self.pause = rospy.ServiceProxy('/gazebo/pause_physics', Empty)
        self.reset_proxy = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)

        self.action_space = spaces.Discrete(3) #F,L,R
        self.reward_range = (-np.inf, np.inf)

        self._seed()

    def discretize_observation(self,data,new_ranges):
        discretized_ranges = []
        min_range = 0.2
        done = False
        mod = len(data.ranges)/new_ranges
        for i, item in enumerate(data.ranges):
            if (i%mod==0):
                if data.ranges[i] == float ('Inf'):
                    discretized_ranges.append(6)
                elif np.isnan(data.ranges[i]):
                    discretized_ranges.append(0)
                else:
                    discretized_ranges.append(int(data.ranges[i]))
            if (min_range > data.ranges[i] > 0):
                done = True
        return discretized_ranges,done

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _step(self, action):

        rospy.wait_for_service('/gazebo/unpause_physics')
        try:
            self.unpause()
        except rospy.ServiceException, e:
            print ("/gazebo/unpause_physics service call failed")

        if action == 0: #FORWARD
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.3
            vel_cmd.angular.z = 0.0
            self.vel_pub.publish(vel_cmd)
        elif action == 1: #LEFT
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.05
            vel_cmd.angular.z = 0.3
            self.vel_pub.publish(vel_cmd)
        elif action == 2: #RIGHT
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.05
            vel_cmd.angular.z = -0.3
            self.vel_pub.publish(vel_cmd)

        data = None
        while data is None:
            try:
                data = rospy.wait_for_message('/kobuki/laser/scan', LaserScan, timeout=5)
            except:
                pass

        rospy.wait_for_service('/gazebo/pause_physics')
        try:
            #resp_pause = pause.call()
            self.pause()
        except rospy.ServiceException, e:
            print ("/gazebo/pause_physics service call failed")

        state,done = self.discretize_observation(data,5)

        if not done:
            if action == 0:
                reward = 5
            else:
                reward = 1
        else:
            reward = -200

        return state, reward, done, {}

    def _reset(self):

        # Resets the state of the environment and returns an initial observation.
        rospy.wait_for_service('/gazebo/reset_simulation')
        try:
            #reset_proxy.call()
            self.reset_proxy()
        except rospy.ServiceException, e:
            print ("/gazebo/reset_simulation service call failed")

        # Unpause simulation to make observation
        rospy.wait_for_service('/gazebo/unpause_physics')
        try:
            #resp_pause = pause.call()
            self.unpause()
        except rospy.ServiceException, e:
            print ("/gazebo/unpause_physics service call failed")

        #read laser data
        data = None
        while data is None:
            try:
                data = rospy.wait_for_message('/kobuki/laser/scan', LaserScan, timeout=5)
            except:
                pass

        rospy.wait_for_service('/gazebo/pause_physics')
        try:
            #resp_pause = pause.call()
            self.pause()
        except rospy.ServiceException, e:
            print ("/gazebo/pause_physics service call failed")

        state = self.discretize_observation(data,5) 

        return state


<p style="background:green;color:white;">**gazebo_circuit_turtlebot_lidar.py**</p>

In this Python file, the **GazeboCircuitTurtlebotLidarEnv(gazebo_env.GazeboEnv)** class is defined.

The main functions of this class are:<br>
<ul>
    <li>**\_\_init\_\_**: Here, all of the topics for communication with ROS are defined. Three of them are for resetting and pausing the simulation. This is used for resetting the simulation when the learning considers that it's in a bad configuration. And the pause is used in each step to simulate in steps and not lose any kind of data that could be generated between learning steps. It also defines a publisher to the **/cmd_vel** topic that will move the robot.<br><br> </li>
    <li>
    **\_step**: Here, you define what will be performed in each learning step. It has as input an **action** variable that contains the action that the learning script considers it has to perform. It's here that each action is really defined. In this case, there are three actions available: **"TURN LEFT,"** **"TURN RIGHT,"** and **"GO FORWARD."**<br><br>
    
    It also defines when the episode will be considered done based on, in this case, the readings of the laser. In this case, an episode is considered done when one of the laser readings is lower than 0.2. This means that the robot is really close to a wall and it is considered to be unsuccessful in accomplishing the learning goal, so a reset is needed to start another learning episode.<br><br>
    Also, there are the rewards for performing each action. These rewards are necessary for the learning algorithms. In this case, the rewards are 5 if it hasn't run into a wall and is going forward, 1 if it hasn't run into a wall and is turning, and -200 if it has run into a wall.<br>    
    </li>
    <li>
    **discretize_observation**: This function cleans up the laser readings. It returns a state variable where there is a discretized version of the laser readings. The laser range readings have got one laser reading for every five real ones. It has also converted Infinite distance readings to 6 and Null reading to 0. All of the other readings have been converted to integers.<br>
    It also returns a done variable that depends on if a certain distance was below the minimum desired.<br><br>
    </li>
    <li>
    **\_reset**: This function will be called each time a reset in the simulation environment is needed, probably because the robot has run into a wall or the time step limit has been reached. It reads the laser data, returning a discretized version through the state variable. It also sends a simulation reset call to the Gazebo simulator.
    </li>
</ul>

<p style="background:#EE9023;color:white;">**Exercise 2.1**</p>

Create a new environment file in you **catkin_ws/src** directory. You can call this file **my_custom_env.py**. You can copy to this file the contents of the <a href="#env-file">**gazebo_circuit_turtlebot_lidar.py**</a> file you have just checked. Now, let's do a couple of modifications so that your training script uses this new environment file.

a) Modify the name of the class in the environment file. For instance, you can call it **MyCustomEnv**.

In [ ]:
# class GazeboCircuitTurtlebotLidarEnv(gazebo_env.GazeboEnv):
class MyCustomEnv(gazebo_env.GazeboEnv):

b) Next, you will have to register your new environment. For that, you will first need to add a register function inside your environment script. It could be something like this:

In [ ]:
reg = register(
    id='MyCustom-v0',
    entry_point='my_custom_env:MyCustomEnv',
    timestep_limit=100,
    )

Add this function right above the class registration.

In order to be able to call this register function, though, you will need to import it first. So, add the following import to your file:

In [ ]:
from gym.envs.registration import register

c) Also, you will have to remove the line where the ROS node is started. This line is the following one:

In [ ]:
gazebo_env.GazeboEnv.__init__(self, "GazeboCircuitTurtlebotLidar_v0.launch")

So, remove or comment this line.

d) Finally, you will have to start this new environment in your learning script. So, open the my_simple_learning_turtlebot.py file, and modify the line where you initialize the environment.

In [ ]:
#env = gym.make('GazeboCircuitTurtlebotLidar-v0')
env = gym.make('MyCustom-v0')

But, in order to be able to use this environment, you will have to import it first. So, add the following import to your file.

In [ ]:
import my_custom_env

e) There's one last thing to do. Start a ROS node in your learning script, right above the line where the environment is started.

In [ ]:
rospy.init_node('turtle_gym', anonymous=True) #This is the line you have to add
env = gym.make('MyCustom-v0')

f) And that's it! Launch your learning script and check that everything works fine.

<p style="background:#EE9023;color:white;">**END of Exercise 2.1**</p>

<p style="background:green;color:white;">Solution Exercise 2.1</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

Here you can see how the final files should look like:

<p style="background:green;color:white;">**my_custom_env.py**</p>

In [ ]:
import gym
import rospy
import roslaunch
import time
import numpy as np

from gym import utils, spaces
from gym_gazebo.envs import gazebo_env
from geometry_msgs.msg import Twist
from std_srvs.srv import Empty

from sensor_msgs.msg import LaserScan

from gym.utils import seeding
from gym.envs.registration import register

reg = register(
    id='MyCustom-v0',
    entry_point='my_custom_env:MyCustomEnv',
    timestep_limit=100,
    )

class MyCustomEnv(gazebo_env.GazeboEnv):

    def __init__(self):
        
        self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=5)
        self.unpause = rospy.ServiceProxy('/gazebo/unpause_physics', Empty)
        self.pause = rospy.ServiceProxy('/gazebo/pause_physics', Empty)
        self.reset_proxy = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
        
        #self.gazebo = GazeboConnection()

        self.action_space = spaces.Discrete(3) #F,L,R
        self.reward_range = (-np.inf, np.inf)

        self._seed()

    def discretize_observation(self,data,new_ranges):
        discretized_ranges = []
        min_range = 0.2
        done = False
        mod = len(data.ranges)/new_ranges
        for i, item in enumerate(data.ranges):
            if (i%mod==0):
                if data.ranges[i] == float ('Inf'):
                    discretized_ranges.append(6)
                elif np.isnan(data.ranges[i]):
                    discretized_ranges.append(0)
                else:
                    discretized_ranges.append(int(data.ranges[i]))
            if (min_range > data.ranges[i] > 0):
                done = True
        return discretized_ranges,done

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _step(self, action):
        rospy.wait_for_service('/gazebo/unpause_physics')
        try:
            self.unpause()
        except rospy.ServiceException, e:
            print ("/gazebo/unpause_physics service call failed")
        if action == 0: #FORWARD
            vel_cmd = Twist()
            vel_cmd.linear.x = 1.0
            vel_cmd.angular.z = 0.0
            self.vel_pub.publish(vel_cmd)
        elif action == 1: #LEFT
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.05
            vel_cmd.angular.z = 0.3
            self.vel_pub.publish(vel_cmd)
        elif action == 2: #RIGHT
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.05
            vel_cmd.angular.z = -0.3
            self.vel_pub.publish(vel_cmd)

        data = None
        while data is None:
            try:
                data = rospy.wait_for_message('/kobuki/laser/scan', LaserScan, timeout=5)
            except:
                print "Time out /kobuki/laser/scan"
                pass
        rospy.wait_for_service('/gazebo/pause_physics')
        try:
            #resp_pause = pause.call()
            self.pause()
        except rospy.ServiceException, e:
            print ("/gazebo/pause_physics service call failed")

        state,done = self.discretize_observation(data,5)

        if not done:
            if action == 0:
                reward = 5
            else:
                reward = 1
        else:
            reward = -200

        return state, reward, done, {}

    def _reset(self):

        # Resets the state of the environment and returns an initial observation.
        rospy.wait_for_service('/gazebo/reset_simulation')
        try:
            #reset_proxy.call()
            self.reset_proxy()
        except rospy.ServiceException, e:
            print ("/gazebo/reset_simulation service call failed")

        # Unpause simulation to make observation
        rospy.wait_for_service('/gazebo/unpause_physics')
        try:
            #resp_pause = pause.call()
            self.unpause()
        except rospy.ServiceException, e:
            print ("/gazebo/unpause_physics service call failed")

        #read laser data
        data = None
        while data is None:
            try:
                data = rospy.wait_for_message('/kobuki/laser/scan', LaserScan, timeout=5)
            except:
                print "Something went wrong reading /kobuki/laser/scan"
                pass

        rospy.wait_for_service('/gazebo/pause_physics')
        try:
            #resp_pause = pause.call()
            self.pause()
        except rospy.ServiceException, e:
            print ("/gazebo/pause_physics service call failed")

        state = self.discretize_observation(data,5) 

        return state



<p style="background:green;color:white;">**my_simple_learning_turtlebot.py**</p>

In [ ]:
#!/usr/bin/env python
import gym
import time
import numpy
import random
import time
import matplotlib
import matplotlib.pyplot as plt
import qlearn
# import liveplot
from gym import wrappers
from liveplot import LivePlot
import rospy

import my_custom_env

def render():
    render_skip = 0 #Skip first X episodes.
    render_interval = 50 #Show render Every Y episodes.
    render_episodes = 10 #Show Z episodes every rendering.

    if (x%render_interval == 0) and (x != 0) and (x > render_skip):
        env.render()
    elif ((x-render_episodes)%render_interval == 0) and (x != 0) and (x > render_skip) and (render_episodes < x):
        env.render(close=True)

if __name__ == '__main__':

    rospy.init_node('turtle_gym', anonymous=True)
    env = gym.make('MyCustom-v0')
    print "Gym Make done"
    outdir = '/tmp/gazebo_gym_experiments'
    #outdir = '/home/user/catkin_ws/src/gym_construct/src/gazebo_gym_experiments'
    # env.monitor.start(outdir, force=True, seed=None)       # I had to comment this and
    env = wrappers.Monitor(env, outdir, force=True)          # use this to avoid warnings
    #plotter = LivePlot(outdir)
    print "Monitor Wrapper started"
    last_time_steps = numpy.ndarray(0)

    qlearn = qlearn.QLearn(actions=range(env.action_space.n),
                    alpha=0.1, gamma=0.8, epsilon=0.9)

    initial_epsilon = qlearn.epsilon

    epsilon_discount = 0.999 # 1098 eps to reach 0.1

    start_time = time.time()
    total_episodes = 10
    highest_reward = 0

    for x in range(total_episodes):
        done = False

        cumulated_reward = 0 #Should going forward give more reward then L/R ?
        print ("Episode = "+str(x))
        observation = env.reset()
        if qlearn.epsilon > 0.05:
            qlearn.epsilon *= epsilon_discount

        #render()
        print "Starting Render"
        env.render()
        print "End Render"
        state = ''.join(map(str, observation))
        max_range = 1000
        for i in range(max_range):

            # Pick an action based on the current state
            action = qlearn.chooseAction(state)
            # Execute the action and get feedback
            observation, reward, done, info = env.step(action)
            cumulated_reward += reward
            if highest_reward < cumulated_reward:
                highest_reward = cumulated_reward

            nextState = ''.join(map(str, observation))

            qlearn.learn(state, action, reward, nextState)

            #env.monitor.flush(force=True)

            if not(done):
                state = nextState
            else:
                print "DONE"
                last_time_steps = numpy.append(last_time_steps, [int(i + 1)])
                break 

        m, s = divmod(int(time.time() - start_time), 60)
        h, m = divmod(m, 60)
        print ("EP: "+str(x+1)+" - [alpha: "+str(round(qlearn.alpha,2))+" - gamma: "+str(round(qlearn.gamma,2))+" - epsilon: "+str(round(qlearn.epsilon,2))+"] - Reward: "+str(cumulated_reward)+"     Time: %d:%02d:%02d" % (h, m, s))

    #Github table content
    print ("\n|"+str(total_episodes)+"|"+str(qlearn.alpha)+"|"+str(qlearn.gamma)+"|"+str(initial_epsilon)+"*"+str(epsilon_discount)+"|"+str(highest_reward)+"| PICTURE |")

    l = last_time_steps.tolist()
    l.sort()

    #print("Parameters: a="+str)
    print("Overall score: {:0.2f}".format(last_time_steps.mean()))
    print("Best 100 score: {:0.2f}".format(reduce(lambda x, y: x + y, l[-100:]) / len(l[-100:])))

    #env.monitor.close()
    #env.close()



<p style="background:green;color:white;">END Solution Exercise 2.1</p>

<p style="background:#EE9023;color:white;">**Exercise 2.2**</p>

Now that everything is working and we have our own environment file, let's do some modifications to it.

<ul>
<li>
Change the rewards given values. For instance, make turning more rewarding than going forward. Or give going forward more weight. Can you see any changes in the emerging behaviour?<br>
</li>
<li>
Change the minimum distance to a higher value to teach the robot to not get so close to walls.
</li>
<li>
Change the speeds of the movements so that it goes faster forward and turns slower.<br>
</li>
<li>
Implement a new action to be performed, and also add it into the learning Python script so that it is performed.
For instance, implement a "GO BACKARDS TURNING" action that is executed when the robot gets too close to a wall. You will also change your own learning script to make it do that new action.<br>
Does it get stuck or in an infinite loop?
</li>
</ul>

<p style="background:#EE9023;color:white;">**END of Exercise 2.2**</p>

### In Local:

Keep in mind that all the topics stated up until now are applicable for the standard gym module locally and the gym-gazebo locally. So, you can perform any of these actions exactly the same in your computer locally, if desired, once you download all of the required software from:

<a href="https://github.com/erlerobot/gym-gazebo">Gym-Gazebo-Git</a><br>
<a href="https://gym.openai.com/docs">Gym-Docs</a>